In [ ]:
import pandas as pd

from pathlib import Path

DATA_PATH = '../../data'
Path(DATA_PATH+'/main').mkdir(parents=True, exist_ok=True)

# Food intake records

Reading file output from AromiDiet, and processing for later analysis.

In [ ]:
#full list of food intakes
#exported from Aromi with options "Tuotteet ja raaka-aineet" and "Fineli ravintoaineet"
aromi = pd.read_excel(
    DATA_PATH+'/RAW/aromi_product_ingredient.xlsx', 
    sheet_name='Tutkimusraportti',
).drop(
    columns=[
        'Syntymäaika',
        'Ryhmätagi',
        'Ruokapäiväkirjaryhmä',
        'Ruokapäiväkirjan nimi',
    ]
)

aromi['timestamp'] = pd.to_datetime (
    aromi.Ruokailuaika
)
aromi.drop(columns=['Ruokailuaika'], inplace=True)

aromi['food_code'] = aromi['Tuotetunnus'].fillna(aromi['Reseptin/tuotteen tunnus'])

## Animal Source Energy Proportion

In [ ]:
#manually created list of animal proportion in food items
fap = pd.read_excel(
    DATA_PATH+'/public/food_animal_proportion.xlsx', 
    sheet_name='manual_entry',
    usecols=('food_code','food_animal_proportion')
)

food_records = aromi.merge(
    fap, 
    left_on='food_code', 
    right_on='food_code', 
    how='left'
).copy() #getting rid of fragmentation warning

food_records['ase'] = food_records.ENERC * food_records.food_animal_proportion
len(food_records.index)

In [ ]:
#food records where energy is empty, not zero
food_records[
    food_records.ENERC.isna()
].groupby(['food_code','Reseptin/tuotteen nimi'])[['Tunnus']].count()

In [ ]:
#food records where food_animal_proportion is empty, not zero
food_records[
    food_records.food_animal_proportion.isna()
].groupby(['food_code','Reseptin/tuotteen nimi'])[['Tunnus']].count()

In [ ]:
#dropping rows where animal source energy value is missing
#the above two check show which codes these are and whether it is because of
# - missing energy data, or
# - missing animal proportion data
intakes = food_records.dropna(subset=['ase'])

In [ ]:
intakes_per_person = intakes.groupby(
    ['Tunnus']
).sum(
    numeric_only=True
).drop(
    columns=[
        'Kulutettu määrä',
        'Käyttömäärä',
        'food_animal_proportion'
    ]
)

intakes_per_person['asep'] = intakes_per_person.ase / intakes_per_person.ENERC
len(intakes_per_person)

## Food record time indicators

In [ ]:
df = food_records[['Tunnus','timestamp']].copy()
df['date'] = df.timestamp.dt.date

fr_dates = df.groupby('Tunnus').agg({'timestamp': ['min', 'max'], 'date': 'nunique'})
fr_dates.columns = ('fr_start','fr_end','fr_days')
#fr_dates['fr_duration'] = fr_dates.fr_end - fr_dates.fr_start

intakes_per_person = intakes_per_person.merge(
    fr_dates,
    left_index=True,
    right_index=True,
)
len(intakes_per_person)

## Normalising intake metrics

In [ ]:
#nutrients are reported as total sums, they need to be expressed as daily intakes

#we are divinding the total intake in the food records by the number of food record days
micro_nutrients = [
    'VITC',
    'F20D5N3',
    'F22D6N3',
    'FE',
    'FOL',
    'CA',
    'F18D2CN6',
    'F18D3N3'
]

micro_means = pd.DataFrame()

for k in micro_nutrients:
    micro_means[f'{k}_daily_mean'] = (intakes_per_person[k] / intakes_per_person.fr_days)

In [ ]:
#energy densities of macronutrients (kJ per 1 g of macronutrient)

e_densities = {
    'FAT': 37,
    'FAPU': 37,
    'FASAT': 37,
    'CHOAVL': 17,
    'PROT': 17,
    'FIBC': 8,
}

e_proportions = pd.DataFrame()

for k,v in e_densities.items():
    # add a column for the total energy from macro nutrient k
    # intakes_per_person[column] contanins intake in grams, v is energy density in kJ/g, result is total energy intake
    e_proportions[f'e-{k}'] = intakes_per_person[k] * v
    # calculcate energy proportion of k
    e_proportions[f'ep-{k}'] = e_proportions[f'e-{k}'] / intakes_per_person.ENERC

In [ ]:
#CHOLE as mg/MJ
CHOLE_per_e = (
    intakes_per_person.CHOLE / (intakes_per_person.ENERC / 1000)
).to_frame(
        name='CHOLE_mg_per_MJ'
) #mg/MJ

In [ ]:
FIBC_per_e = (
    intakes_per_person.FIBC / (intakes_per_person.ENERC / 1000)
).to_frame(
        name='FIBC_g_per_MJ'
) #g/MJ

In [ ]:
intakes_per_person = pd.concat(
    [
        intakes_per_person,
        micro_means,
        e_proportions,
        CHOLE_per_e,
        FIBC_per_e
    ],
    axis=1
)

## Exporting to excel file

In [ ]:
intakes_per_person.index.names = ['id']

intakes_per_person.to_excel(
    DATA_PATH+'/main/intakes_detailed_with_asep.xlsx'
)